# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [4]:
problem = '''Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.  
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment. 

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are four possible budget scenarios with different probabilities: scenario 1 with 1000 euros and probability of 0.55, scenario 2  with 1100 euros and probability of 0.4, scenario 3 with 900 euros and probability of 0.04, scenario 4 with 1200 euros and probability of 0.01. 
The tolerable probability of exceeding the budget is 0.4.

Please formulate a mean-variance mathematical model for this optimization problem, considering the past performance of investment strategies and the uncertain budget. You can take 2 as the risk parameter r.
'''

## 2. Generate the mathematical model

In [5]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [6]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mean-Variance Optimization Model with Uncertain Budget

### Sets and Indices:

* $S$: Set of investment strategies, $S = \{1, 2, ..., 10\}$
* $B$: Set of budget scenarios, $B = \{1, 2, 3, 4\}$ 

### Parameters:

* $r_s$: Historical return of strategy $s \in S$ (from "investments_data.csv")
* $c_s$: Cost of investing in strategy $s \in S$, where $c = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]$
* $b_j$: Budget amount in scenario $j \in B$, where $b = [1000, 1100, 900, 1200]$
* $p_j$: Probability of budget scenario $j \in B$, where $p = [0.55, 0.4, 0.04, 0.01]$
* $R$: Risk parameter, $R = 2$
* $\alpha$: Tolerable probability of exceeding the budget, $\alpha = 0.4$

### Decision Variables:

* $x_s$: Binary variable, 1 if strategy $s$ is chosen, 0 otherwise. 

### Objective Function:

Maximize the expected return minus a penalty for exceeding the budget:

```
Maximize  ∑_(s∈S) r_s * x_s - R * ∑_(j∈B) p_j * max(0, ∑_(s∈S) c_s * x_s - b_j)
```

* The first term calculates the expected return based on the chosen strategies and their historical returns.
* The second term penalizes exceeding the budget in each scenario, weighted by the scenario probability and the risk parameter.

### Constraints:

1. **Budget Constraint:** Limit the probability of exceeding the budget:

```
∑_(j∈B) p_j * I(∑_(s∈S) c_s * x_s > b_j) <= α
```

* $I(\cdot)$ is an indicator function, equal to 1 if the condition inside is true, and 0 otherwise. This constraint ensures the probability of exceeding the budget across all scenarios remains below the tolerable level $\alpha$.

2. **Investment Choice:** Invest in each strategy at most once:

```
x_s ∈ {0, 1}  ∀ s ∈ S
```

### Model Interpretation:

This model aims to find the optimal investment strategy combination that maximizes the expected return while controlling the risk of exceeding the budget under different scenarios. The objective function balances maximizing return and minimizing potential budget overruns. The constraints ensure the solution remains within the acceptable risk tolerance and adheres to the investment limitations.

**Note:** Solving this model requires a mixed-integer programming solver due to the binary decision variables.  


## 3. Generate the pyomo code

In [7]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [8]:
#Showing the code in a formatted way
Markdown(response2.text)

## Pyomo Implementation of Mean-Variance Optimization with Uncertain Budget

```python
from pyomo.environ import *

# Sample data (replace with actual data from investments_data.csv)
investments_data = {
    's': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'r': [0.12, 0.08, 0.15, 0.05, 0.10, 0.09, 0.11, 0.13, 0.07, 0.14],
}
c = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]  # Sample cost data
b = [1000, 1100, 900, 1200]  # Sample budget data
p = [0.55, 0.4, 0.04, 0.01]  # Sample probability data
R = 2  # Risk parameter
alpha = 0.4  # Tolerable probability of exceeding the budget

# Define the model
model = ConcreteModel()

# Sets and indices
model.S = Set(initialize=investments_data['s'])
model.B = Set(initialize=range(1, len(b) + 1))

# Parameters
model.r = Param(model.S, initialize={s: investments_data['r'][i] for i, s in enumerate(investments_data['s'])})
model.c = Param(model.S, initialize={s: c[i] for i, s in enumerate(investments_data['s'])})
model.b = Param(model.B, initialize={j: b[j - 1] for j in model.B})
model.p = Param(model.B, initialize={j: p[j - 1] for j in model.B})
model.R = Param(initialize=R)
model.alpha = Param(initialize=alpha)

# Decision variables
model.x = Var(model.S, domain=Binary)

# Objective function
def objective_rule(model):
    return sum(model.r[s] * model.x[s] for s in model.S) - model.R * sum(
        model.p[j] * max(0, sum(model.c[s] * model.x[s] for s in model.S) - model.b[j]) for j in model.B
    )

model.objective = Objective(rule=objective_rule, sense=maximize)

# Constraints
def budget_constraint_rule(model, j):
    return sum(model.p[j] * (sum(model.c[s] * model.x[s] for s in model.S) > model.b[j]) for j in model.B) <= model.alpha

model.budget_constraint = Constraint(model.B, rule=budget_constraint_rule)

# Solve the model
solver = SolverFactory('cbc')  # Replace with your preferred solver
solver.solve(model)

# Print the results
print("Optimal solution:")
for s in model.S:
    if model.x[s].value > 0.5:
        print(f"Invest in strategy {s}")

print(f"Expected return: {model.objective()}")
```

**Explanation:**

1. **Import Pyomo:** We import the necessary Pyomo library.
2. **Sample Data:** We define sample data for returns (replace with actual data), costs, budgets, probabilities, risk parameter, and tolerable probability.
3. **Model Definition:** We create a ConcreteModel instance to represent the optimization problem.
4. **Sets and Indices:** We define sets for strategies (S) and budget scenarios (B).
5. **Parameters:** We define parameters for returns, costs, budgets, probabilities, risk parameter, and tolerable probability, initializing them with the sample data.
6. **Decision Variables:** We define binary decision variables x[s] to represent whether to invest in strategy s or not.
7. **Objective Function:** We define the objective function using a rule that calculates the expected return minus the penalty for exceeding the budget.
8. **Constraints:** We define the budget constraint using a rule that ensures the probability of exceeding the budget across all scenarios remains below the tolerable level.
9. **Solver:** We choose a solver (e.g., CBC) to solve the mixed-integer programming problem.
10. **Results:** We print the optimal investment decisions and the expected return. 


## 4. Input problem data and try running the generated code

In [9]:
from pyomo.environ import *

# Sample data (replace with actual data from investments_data.csv)
investments_data = {
    's': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'r': [0.12, 0.08, 0.15, 0.05, 0.10, 0.09, 0.11, 0.13, 0.07, 0.14],
}
c = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]  # Sample cost data
b = [1000, 1100, 900, 1200]  # Sample budget data
p = [0.55, 0.4, 0.04, 0.01]  # Sample probability data
R = 2  # Risk parameter
alpha = 0.4  # Tolerable probability of exceeding the budget

# Define the model
model = ConcreteModel()

# Sets and indices
model.S = Set(initialize=investments_data['s'])
model.B = Set(initialize=range(1, len(b) + 1))

# Parameters
model.r = Param(model.S, initialize={s: investments_data['r'][i] for i, s in enumerate(investments_data['s'])})
model.c = Param(model.S, initialize={s: c[i] for i, s in enumerate(investments_data['s'])})
model.b = Param(model.B, initialize={j: b[j - 1] for j in model.B})
model.p = Param(model.B, initialize={j: p[j - 1] for j in model.B})
model.R = Param(initialize=R)
model.alpha = Param(initialize=alpha)

# Decision variables
model.x = Var(model.S, domain=Binary)

# Objective function
def objective_rule(model):
    return sum(model.r[s] * model.x[s] for s in model.S) - model.R * sum(
        model.p[j] * max(0, sum(model.c[s] * model.x[s] for s in model.S) - model.b[j]) for j in model.B
    )

model.objective = Objective(rule=objective_rule, sense=maximize)

# Constraints
def budget_constraint_rule(model, j):
    return sum(model.p[j] * (sum(model.c[s] * model.x[s] for s in model.S) > model.b[j]) for j in model.B) <= model.alpha

model.budget_constraint = Constraint(model.B, rule=budget_constraint_rule)

# Solve the model
solver = SolverFactory('cbc')  # Replace with your preferred solver
solver.solve(model)

# Print the results
print("Optimal solution:")
for s in model.S:
    if model.x[s].value > 0.5:
        print(f"Invest in strategy {s}")

print(f"Expected return: {model.objective()}")

ERROR: Rule failed when generating expression for Objective objective with
index None: PyomoException: Cannot convert non-constant Pyomo expression (0  <
80*x[1] + 340*x[2] + 410*x[3] + 50*x[4] + 180*x[5] + 221*x[6] + 15*x[7] +
348*x[8] + 191*x[9] + 225*x[10] - 1000) to bool. This error is usually caused
by using a Var, unit, or mutable Param in a Boolean context such as an "if"
statement, or when checking container membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'objective' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression (0  <
        80*x[1] + 340*x[2] + 410*x[3] + 50*x[4] + 180*x[5] + 221*x[6] +
        15*x[7] + 348*x[8] + 191*x[9] + 225*x[10] - 1000) to bool.
    This error is usually caused by using a Var, unit, or mutable Para

PyomoException: Cannot convert non-constant Pyomo expression (0  <  80*x[1] + 340*x[2] + 410*x[3] + 50*x[4] + 180*x[5] + 221*x[6] + 15*x[7] + 348*x[8] + 191*x[9] + 225*x[10] - 1000) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [10]:
print(response.text)

## Mean-Variance Optimization Model with Uncertain Budget

### Sets and Indices:

* $S$: Set of investment strategies, $S = \{1, 2, ..., 10\}$
* $B$: Set of budget scenarios, $B = \{1, 2, 3, 4\}$ 

### Parameters:

* $r_s$: Historical return of strategy $s \in S$ (from "investments_data.csv")
* $c_s$: Cost of investing in strategy $s \in S$, where $c = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]$
* $b_j$: Budget amount in scenario $j \in B$, where $b = [1000, 1100, 900, 1200]$
* $p_j$: Probability of budget scenario $j \in B$, where $p = [0.55, 0.4, 0.04, 0.01]$
* $R$: Risk parameter, $R = 2$
* $\alpha$: Tolerable probability of exceeding the budget, $\alpha = 0.4$

### Decision Variables:

* $x_s$: Binary variable, 1 if strategy $s$ is chosen, 0 otherwise. 

### Objective Function:

Maximize the expected return minus a penalty for exceeding the budget:

```
Maximize  ∑_(s∈S) r_s * x_s - R * ∑_(j∈B) p_j * max(0, ∑_(s∈S) c_s * x_s - b_j)
```

* The first term calculates the expected

In [11]:
print(response2.text)

## Pyomo Implementation of Mean-Variance Optimization with Uncertain Budget

```python
from pyomo.environ import *

# Sample data (replace with actual data from investments_data.csv)
investments_data = {
    's': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'r': [0.12, 0.08, 0.15, 0.05, 0.10, 0.09, 0.11, 0.13, 0.07, 0.14],
}
c = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]  # Sample cost data
b = [1000, 1100, 900, 1200]  # Sample budget data
p = [0.55, 0.4, 0.04, 0.01]  # Sample probability data
R = 2  # Risk parameter
alpha = 0.4  # Tolerable probability of exceeding the budget

# Define the model
model = ConcreteModel()

# Sets and indices
model.S = Set(initialize=investments_data['s'])
model.B = Set(initialize=range(1, len(b) + 1))

# Parameters
model.r = Param(model.S, initialize={s: investments_data['r'][i] for i, s in enumerate(investments_data['s'])})
model.c = Param(model.S, initialize={s: c[i] for i, s in enumerate(investments_data['s'])})
model.b = Param(model.B, initialize={j: b[